# Convert Coco JSON Annotations to YOLO TXT Files
Use this notebook to try out importing, analyzing, and exporting datasets of image annotations. 

### NB! Make sure to have renamed images and img_filename in annotations to correct format first

In [1]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)
import sys
!{sys.executable} -m pip install pylabel
from pylabel import importer

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


# YOLO Format

All the bounding boxes are placed in a `.txt`-file for each image in the dataset.

The order of attributes in each row is ```[class x_centered y_centered width height]```, where all of the values are normalized in the range of [0...1]

The centered values is the center values in both width (x) and height (y) direction, and the width and height tell the size of the bounding box.

An example of such a file:
```
0 0.0172 0.8358 0.0315 0.0347
0 0.1301 0.9474 0.0441 0.0490
0 0.1054 0.9319 0.0936 0.0891
0 0.0275 0.9392 0.0308 0.0311
```


## Import coco annotations 
First we will import annotations from the coco dataset, which are in coco json format. 

In [2]:
#Specify path to the coco.json file and images, if in another folder than annotations
TRAIN = False
dataset_path = "../datasets/tdt4265"
if TRAIN:
    path_to_annotations = "./original_annotations_json/train_annotations.json"
    path_to_images = os.path.join(dataset_path, "images/train")
else:
    path_to_annotations = "./original_annotations_json/val_annotations.json"
    path_to_images = os.path.join(dataset_path, "images/train")

#Import the dataset into the pylable schema 
dataset = importer.ImportCoco(path_to_annotations, path_to_images=path_to_images, name="labeled_data_coco_format")
dataset.df.head(1)

,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_segmentation,ann_iscrowd,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,../datasets/tdt4265/images/train,images/val/trip007_glos_Video00003_54.png,,1,1024,128,,,1020.42,65.4,...,[],0,,,,1,car,,,1


## Analyze annotations
Pylabel can calculate basic summary statisticts about the dataset such as the number of files and the classes. 
The dataset is stored as a pandas frame so the developer can do additional exploratory analysis on the dataset. 

In [3]:
print(f"Number of images: {dataset.analyze.num_images}")
print(f"Number of classes: {dataset.analyze.num_classes}")
print(f"Classes:{dataset.analyze.classes}")
print(f"Class counts:\n{dataset.analyze.class_counts}")
print(f"Path to annotations:\n{dataset.path_to_annotations}")


Number of images: 301
Number of classes: 4
Classes:['car', 'bus', 'person', 'rider']
Class counts:
person    3699
car       1437
rider      315
bus         93
Name: cat_name, dtype: int64
Path to annotations:
original_annotations_json


## Visualize Annotations 
You can render the bounding boxes for your image to inspect them and confirm that they imported correctly.  

### Fikser sentrering av bokser

In [4]:
data = [[] for _ in range(4)]
for index, row in dataset.df.iterrows():
    xmin = min(row["ann_bbox_xmin"], row["ann_bbox_xmax"])
    ymin = min(row["ann_bbox_ymin"], row["ann_bbox_ymax"])
    data[0].append(xmin + (abs(row["ann_bbox_xmin"] - row["ann_bbox_xmax"])/2))
    data[1].append(ymin + (abs(row["ann_bbox_ymin"] - row["ann_bbox_ymax"])/2))
    data[2].append((abs(row["ann_bbox_xmin"] - row["ann_bbox_xmax"])))
    data[3].append((abs(row["ann_bbox_ymin"] - row["ann_bbox_ymax"])))

dataset.df["center_x_scaled"] = data[0]
dataset.df["center_y_scaled"] = data[1]
dataset.df["width_scaled"] = data[2]
dataset.df["height_scaled"] = data[3]
dataset.df["center_x_scaled"]

id
0       1022.210
1        984.505
2        543.810
3        474.105
4        472.445
          ...   
5539      17.515
5540      57.900
5541      79.305
5542     136.820
5543     605.575
Name: center_x_scaled, Length: 5544, dtype: float64

In [5]:
img_id = 1 #0
selected_image_df = dataset.df.loc[dataset.df['img_id'] == img_id]
print(selected_image_df)

                          img_folder  \
id                                     
0   ../datasets/tdt4265/images/train   
1   ../datasets/tdt4265/images/train   
2   ../datasets/tdt4265/images/train   
3   ../datasets/tdt4265/images/train   
4   ../datasets/tdt4265/images/train   
5   ../datasets/tdt4265/images/train   
6   ../datasets/tdt4265/images/train   
7   ../datasets/tdt4265/images/train   
8   ../datasets/tdt4265/images/train   
9   ../datasets/tdt4265/images/train   
10  ../datasets/tdt4265/images/train   
11  ../datasets/tdt4265/images/train   
12  ../datasets/tdt4265/images/train   
13  ../datasets/tdt4265/images/train   
14  ../datasets/tdt4265/images/train   
15  ../datasets/tdt4265/images/train   
16  ../datasets/tdt4265/images/train   
17  ../datasets/tdt4265/images/train   
18  ../datasets/tdt4265/images/train   

                                 img_filename img_path  img_id  img_width  \
id                                                                          
0   i

In [6]:
import cv2
import os

# Open and resize image
img = cv2.imread(os.path.join(dataset_path, selected_image_df["img_filename"][0]))
WIDTH = selected_image_df["img_width"][0]
HEIGHT = selected_image_df["img_height"][0]
print(WIDTH, HEIGHT)

# Print bounding boxes
for index, row in selected_image_df.iterrows():
    # x = int(row["center_x_scaled"]*WIDTH-row["width_scaled"]*WIDTH/2)
    # y = int(row["center_y_scaled"]*HEIGHT-row["height_scaled"]*HEIGHT/2)
    # w = int(row["width_scaled"]*WIDTH)
    # h = int(row["height_scaled"]*HEIGHT)
    # cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
    x = int(row["ann_bbox_xmin"])
    y = int(row["ann_bbox_ymin"])
    w = int(row["ann_bbox_xmax"])
    h = int(row["ann_bbox_ymax"])
    cv2.rectangle(img, (x, y), (w, h), (0, 0, 255), 2)
cv2.imwrite(f"./labeled_im{img_id}.png", img)

1024 128


[ WARN:0@1.385] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../datasets/tdt4265/images/val/trip007_glos_Video00003_54.png'): can't open/read file: check file path/integrity


error: OpenCV(4.5.5) /io/opencv/modules/imgcodecs/src/loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'


In [ ]:
dataset.df.head(5)

# Export to Yolo v5
The PyLabel exporter will export all of the annotations in the dataframe to the desired target format.
Yolo creates one text file for each image in the dataset. 

In [ ]:
dataset.export.ExportToYoloV5(output_path="labels/")